In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import cartopy.feature as cfeature
import pandas as pd
import datetime as dt
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
ds=xr.open_dataset(dir+'station_numbers_95.nc')#load chosen stations and associated info
print(ds)
lats1=ds['lats']
lons1=ds['lons']
elevs=ds['elevs']
stations1=ds['stations']
thresholds=ds['thresholds_annual_99']/10.
print(stations1)

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e

from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(20,16))
ax=plt.subplot(1,1,1,projection=proj_map)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)

ax.set_extent([-84,-66,36,48],crs=ccrs.PlateCarree())

# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-90,-85,-80,-75,-70,-60,-50]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0)
ax.tick_params(labelsize=24)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
ax.scatter(lons1,lats1,s=120,transform=ccrs.PlateCarree(),c='k')
ax.set_title('Eligible GHCND Stations',fontsize=36)
#for i in range(len(station_names.values.tolist())):
#    ax.text(lons[i]+0.1,lats[i],station_names.values[i],transform=ccrs.PlateCarree(),fontsize=8)
plt.show()

In [ ]:
fig.savefig(dir+'eligible_ghcnd_stations_95.png')

In [ ]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            years.append(year)
                            months.append(month)
                            days1.append(i)
                            observations.append(obs/10.)#convert from tenths of mm to mm
                            lats1.append(lat)
                            lons1.append(lon)
                            elevs1.append(elev)
                            date=dt.datetime(year,month,i+1,6)
                            dates.append(date)
                            station_names_list.append(station_name)
                            stations_list.append(station)
#lons1=[x+360. for x in lons1]

In [ ]:
ds.close()
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99_80_annual.nc')#annual here is actually fall, just with annual threshold 
#didn't intend to use annual 99% threshold for station precipitation, but didn't realize that had happened
#until the project was done, and since the trends were demonstrated to be relatively consistent across
#different thresholds, it should not significantly affect results.  Sensitivity testing is shown in the sensitivity
#testing script.

lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
print(len(dates))
zipped=list(zip(lats,lons,dates,stations,obs))
print(len(zipped))

In [ ]:
#single day here
date=dt.datetime(1993,11,28,6)
#date=dates_unique[i]
zipped1=[x for x in zipped if pd.to_datetime(x[2])==date]
obs_mean_list=[]
for i in range(len(stations1)):
    print(i)
    station=stations1[i]
    select_station=[x for x in zipped1 if x[3]==station]
    select_obs=[x[4] for x in select_station]

    if len(select_obs)>0:
        obs_mean=select_obs[0]
        obs_mean_list.append(obs_mean)
    else:
        obs_mean_list.append(np.nan)

In [ ]:
#average of multiple days here
obs_mean_list=[]
for i in range(len(stations1)):
    station=stations1[i]
    zipped1=[x for x in zipped if x[3]==station and x[2] in dates_unique]
    print(len(zipped1))
    if len(zipped1)>0:
        obs_select=[x[4] for x in zipped1]
        mean_obs=float(sum(obs_select))/float(len(obs_select))
        obs_mean_list.append(mean_obs)

In [ ]:
if 1==1:#Now plot composite of extreme days
    clon=-70
    clat=35
    proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
    fig = plt.figure(figsize=(16.6,16))
    ax=plt.subplot(1,1,1,projection=proj_map)

    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
    ax.add_feature(cfeature.LAKES.with_scale('50m'))
    countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
    ax.add_feature(countries)
    ax.set_extent([-84,-66,36,47.65],crs=ccrs.PlateCarree())

    # *must* call draw in order to get the axis boundary used to add ticks:
    fig.canvas.draw()

    if 1==0:
        # Define gridline locations and draw the lines using cartopy's built-in gridliner:
        xticks = [-90,-85,-80,-75,-70,-65,-60,-50]
        yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
        ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0)
        ax.tick_params(labelsize=26)
        # Label the end-points of the gridlines using the custom tick makers:
        ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
        ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
        lambert_xticks(ax, xticks)
        lambert_yticks(ax, yticks)
    cax=ax.scatter(lons1,lats1,s=240,c=obs_mean_list,transform=ccrs.PlateCarree(),vmin=0,vmax=45,cmap=plt.cm.YlGnBu)
    #cax=ax.scatter(lons1,lats1,s=240,c=obs_mean_list,transform=ccrs.PlateCarree(),vmin=0,vmax=150,cmap=plt.cm.Greens)
   
    #cbar=plt.colorbar(cax,pad=0,extend='max',fraction=0.046)
    cbar=plt.colorbar(cax,pad=0,extend='max',orientation='horizontal',aspect=50,fraction=0.04,ticks=[20,40,60,80,100,120,140])
    cbar.ax.tick_params(labelsize=26)
    cbar.set_label('mm/day',fontsize=24,labelpad=15)
    ax.set_title('Average Extreme Precipitation Day',fontsize=36)
    #ax.set_title('November 28, 1993 daily precipitation',fontsize=32)
    #props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
    #ax.text(0.01, 0.99,'n=314', transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props)
    
    plt.show()

In [ ]:
#fig.savefig(dir+'extreme_day_composite_ghcnd_95.png')
fig.savefig(dir+'daily_precip_case6_1a.png')

In [ ]:
#now plot 99% threshold
ds=xr.open_dataset(dir+'station_numbers_95.nc')
lats=ds['lats']
lons=ds['lons']
thresholds=ds['thresholds_annual_99']/10.#still in tenths of mm here.  Check later
thresholds_winter=ds['thresholds_winter_99']/10.
thresholds_spring=ds['thresholds_spring_99']/10.
thresholds_summer=ds['thresholds_summer_99']/10.
thresholds_fall=ds['thresholds_fall_99']/10.

elevs=ds['elevs']
station_names=ds['station_names']

if 1==1:
    clon=-70
    clat=35
    proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
    fig = plt.figure(figsize=(20,16))
    ax=plt.subplot(1,1,1,projection=proj_map)

    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
    ax.add_feature(cfeature.LAKES.with_scale('50m'))
    countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
    ax.add_feature(countries)
    ax.set_extent([-84,-66,36,48],crs=ccrs.PlateCarree())

    # *must* call draw in order to get the axis boundary used to add ticks:
    fig.canvas.draw()

    # Define gridline locations and draw the lines using cartopy's built-in gridliner:
    xticks = [-90,-85,-80,-75,-70,-65,-60,-50]
    yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
    ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0)
    ax.tick_params(labelsize=28)
    # Label the end-points of the gridlines using the custom tick makers:
    ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
    ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
    lambert_xticks(ax, xticks)
    lambert_yticks(ax, yticks)
    cax=ax.scatter(lons,lats,s=240,c=thresholds,transform=ccrs.PlateCarree(),vmin=30,vmax=80,cmap=plt.cm.YlGnBu)
    cbar=plt.colorbar(cax,pad=0,extend='both',fraction=0.046)
    cbar.ax.tick_params(labelsize=32)
    cbar.set_label('mm/day',fontsize=32,rotation=90,labelpad=15)
    ax.set_title('99% Threshold',fontsize=46)
    #ax.set_title('Winter',fontsize=46,pad=15)
    plt.show()

In [ ]:
fig.savefig(dir+'neusa_threshold_seasonal_20.png')